In [3]:
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np

import cv2

from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import numpy as np


In [4]:
import cv2
import mediapipe as mp
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

hand_landmarks_dict = {
    0: "WRIST",
    1: "THUMB_CMC",
    2: "THUMB_MCP",
    3: "THUMB_IP",
    4: "THUMB_TIP",
    5: "INDEX_FINGER_MCP",
    6: "INDEX_FINGER_PIP",
    7: "INDEX_FINGER_DIP",
    8: "INDEX_FINGER_TIP",
    9: "MIDDLE_FINGER_MCP",
    10: "MIDDLE_FINGER_PIP",
    11: "MIDDLE_FINGER_DIP",
    12: "MIDDLE_FINGER_TIP",
    13: "RING_FINGER_MCP",
    14: "RING_FINGER_PIP",
    15: "RING_FINGER_DIP",
    16: "RING_FINGER_TIP",
    17: "PINKY_MCP",
    18: "PINKY_PIP",
    19: "PINKY_DIP",
    20: "PINKY_TIP"
}

# 손 랜드마크 추출 함수
def get_hand_landmarks(frame, hands):
    image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    result = hands.process(image_rgb)
    if result.multi_hand_landmarks:
        landmarks = []
        for hand_landmarks in result.multi_hand_landmarks:
            for lm in hand_landmarks.landmark:
                landmarks.append([lm.x, lm.y, lm.z])
        return np.array(landmarks)
    return None

# Cosine Similarity 계산 함수
def calculate_similarity(hand1, hand2):
    if hand1 is not None and hand2 is not None:
        # 유사도 계산
        hand1_flat = hand1.flatten().reshape(1, -1)
        hand2_flat = hand2.flatten().reshape(1, -1)
        similarity = cosine_similarity(hand1_flat, hand2_flat)[0][0]
        return similarity
    return 0


### 타겟 비디오의 손 관절 측정


In [5]:
# 색상 팔레트
colors={
    "purple_1":[120, 58, 221],
    "purple_2":[147, 97, 228],
    "purple_3":[174, 137, 235],
    "purple_4":[201, 176, 241],
    "purple_5":[228, 216, 248]
}

### test code

In [21]:
# 비디오 파일 경로
video_path = "../demo.mp4"  # 여기에 비디오 파일 경로를 입력하세요
cap = cv2.VideoCapture(video_path)

# 첫 번째 프레임 읽기
ret, first_frame = cap.read()
mp_hands = mp.solutions.hands
target_rgb = cv2.cvtColor(first_frame, cv2.COLOR_BGR2RGB)
target_result = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.7).process(target_rgb)

I0000 00:00:1724217578.096784       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1


In [25]:
target_result.multi_hand_landmarks[:3]

[landmark {
   x: 0.31285524
   y: 0.43669352
   z: -8.1501355e-08
 }
 landmark {
   x: 0.35576642
   y: 0.37872756
   z: -0.012643154
 }
 landmark {
   x: 0.40838283
   y: 0.35572007
   z: -0.023554483
 }
 landmark {
   x: 0.45024687
   y: 0.32532418
   z: -0.031644855
 }
 landmark {
   x: 0.47279212
   y: 0.27862054
   z: -0.040059034
 }
 landmark {
   x: 0.42405644
   y: 0.46439064
   z: -0.031416975
 }
 landmark {
   x: 0.48579523
   y: 0.47447377
   z: -0.043217737
 }
 landmark {
   x: 0.5208652
   y: 0.48485962
   z: -0.05002144
 }
 landmark {
   x: 0.5482905
   y: 0.49622017
   z: -0.05426477
 }
 landmark {
   x: 0.41791302
   y: 0.5267637
   z: -0.025718706
 }
 landmark {
   x: 0.46657196
   y: 0.51197946
   z: -0.03395568
 }
 landmark {
   x: 0.45968688
   y: 0.478575
   z: -0.02877103
 }
 landmark {
   x: 0.44391435
   y: 0.46422487
   z: -0.023873968
 }
 landmark {
   x: 0.4046017
   y: 0.5651892
   z: -0.019198706
 }
 landmark {
   x: 0.44566756
   y: 0.5436389
   z: -0.024

# mediapipe

In [1]:
import cv2
import mediapipe as mp
import time

# Mediapipe 손 모델 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# 색상 및 두께 지정
landmark_drawing_spec = mp_drawing.DrawingSpec(color=colors["purple_2"], thickness=2) 
connection_drawing_spec = mp_drawing.DrawingSpec(color=colors["purple_1"], thickness=2) 


# 비디오 파일 경로
video_path = "../demo.mp4"  # 여기에 비디오 파일 경로를 입력하세요
target_cap = cv2.VideoCapture(video_path)

web_cap = cv2.VideoCapture(0)
# 첫 번째 프레임 읽기
ret, first_frame = cap.read()

start_time = time.time()
while(True):
    # 해상도 설정
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    print(f'현재 해상도: {frame_width}x{frame_height}', f'현재 fps:{fps}')

    web_cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # 가로 해상도
    web_cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 360)  # 세로 해상도

    # 프레임 속도 설정 (예: 30 FPS)
    web_cap.set(cv2.CAP_PROP_FPS, 30)  # 초당 프레임 수
    
    with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
        while web_cap.isOpened():
            target_ret, target_frame = target_cap.read()
            web_ret, web_frame = web_cap.read()
            if not web_ret:
                break
            if not target_ret:
                break

            # 타겟 손 랜드마크 추출
            target_rgb = cv2.cvtColor(target_frame, cv2.COLOR_BGR2RGB)
            web_rgb = cv2.cvtColor(web_frame, cv2.COLOR_BGR2RGB)
            target_result = hands.process(target_rgb)
            web_result = hands.process(web_rgb)
            
            current_time = time.time()
                 
            # 재활자의 랜드마크 표시
            if web_result.multi_hand_landmarks:
                for web_landmarks in web_result.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(web_frame, web_landmarks, mp_hands.HAND_CONNECTIONS,
                                            landmark_drawing_spec,
                                            connection_drawing_spec)
                    
            
            # # 타겟 랜드마크 표시
            # if target_result.multi_hand_landmarks:
            #     for target_landmarks in target_result.multi_hand_landmarks:
            #         # mp_drawing.draw_landmarks(target_frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            #         mp_drawing.draw_landmarks(web_frame, target_landmarks, mp_hands.HAND_CONNECTIONS)
                    
                    
            # 운동 시간 출력   
            elapsed_time = time.time() - start_time
            
            # 경과 시간을 분:초 형식으로 변환
            hours = int(elapsed_time//3600)
            minutes = int(elapsed_time // 60)
            seconds = int(elapsed_time % 60)
            time_text = f'Time: :{minutes:02}:{seconds:02}'     
            cv2.putText(web_frame, time_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

            
            # 화면에 프레임 표시
            cv2.imshow('Hand Tracking', web_frame)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                target_cap.release()
                cv2.destroyAllWindows()
                exit()
    target_cap.release()
cv2.destroyAllWindows()


end_time=time.time()

NameError: name 'colors' is not defined

### 입력된 비디오의 두손 유사도

In [7]:
# Mediapipe 손 모델 초기화
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils

# 비디오 캡처 초기화
target_cap = cv2.VideoCapture(0)

with mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.7) as hands:
    while target_cap.isOpened():
        target_ret, target_frame = target_cap.read()
        if not target_ret:
            break

        # 손 랜드마크 추출
        hand_landmarks = get_hand_landmarks(target_frame, hands)

        # 손 유사도 계산 (두 손이 모두 인식된 경우)
        if hand_landmarks is not None and len(hand_landmarks) == 42:  # 두 손 모두 21개 랜드마크씩 총 42개
            hand1 = hand_landmarks[:21]
            hand2 = hand_landmarks[21:]
            similarity = calculate_similarity(hand1, hand2)
            cv2.putText(target_frame, f'Similarity: {similarity:.2f}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

        # 랜드마크 시각화
        # if hand_landmarks is not None:
        #     for hand_landmark in hand_landmarks.reshape(-1, 21, 3):
        #         mp_drawing.draw_landmarks(frame, mp_hands.HandLandmarkList(landmark=hand_landmark))

        cv2.imshow('Hand Tracking', target_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

target_cap.release()
cv2.destroyAllWindows()

I0000 00:00:1724215070.474400       1 gl_context.cc:344] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M1
